# Technical Report

## Methodology 

I read the article 'Analyzing COVID-related Stance and Arguments using BERT-based Natural Language Inference' and according to it decided to translate text to english to use BERT covid moedel to get the embeddings. I translated, removed stopwords and numbers, then got the embeddings in shape [42,1024] and normalized it. After it I used MLP for classification task, I also changed -1 label to 3 to not to ruin MLP work.

## Discussion of results

As for embeddings part of the task, I think that i did well and did everything correctly. But as for classification task part, I think I should have chosen another model, just because MLP just overfits from the beginning. But it still gives the results above the baseline, however it is difficult to get results lower according to the amount of -1 labels) 

# Code

## Requirements

In [1]:
!pip install transformers sentencepiece sacrebleu datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━

Import necessary libraries

In [14]:
pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-28sglb23
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-28sglb23
  Resolved https://github.com/huggingface/accelerate to commit dcde1e93d09abea02a8e7f4a07a2c5734b87b60e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.20.0.dev0-py3-none-any.whl size=222965 sha256=d14fe1624d52171983729547ebab0547348667a56dab4c79121ab408cb7018e4
  Stored in directory: /tmp/pip-ephem-wheel-cache-dwkch5ch/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.19.0
    Uninstalling accelerate-0.19.0:
      Succe

In [1]:
import os
from tqdm.auto import tqdm

import pandas as pd
import torch
from tqdm import tqdm
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    AutoTokenizer,
    AutoModelForSeq2SeqLM
)
from torch.utils.data import DataLoader
from datasets import Dataset

## Download the data

In [2]:
!wget https://raw.githubusercontent.com/s-nlp/russe_detox_2022/main/data/input/train.tsv
!wget https://raw.githubusercontent.com/s-nlp/russe_detox_2022/main/data/input/test.tsv

--2023-05-16 08:51:18--  https://raw.githubusercontent.com/s-nlp/russe_detox_2022/main/data/input/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1902888 (1.8M) [text/plain]
Saving to: ‘train.tsv.1’

train.tsv.1         100%[===================>]   1.81M  --.-KB/s    in 0.007s  

2023-05-16 08:51:18 (250 MB/s) - ‘train.tsv.1’ saved [1902888/1902888]

--2023-05-16 08:51:18--  https://raw.githubusercontent.com/s-nlp/russe_detox_2022/main/data/input/test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104462 (102K) [tex

In [3]:
train_df = pd.read_csv('train.tsv', sep = '\t')
test_df = pd.read_csv('test.tsv', sep = '\t')

In [4]:
train_df = train_df.drop(['neutral_comment2', 'neutral_comment3', 'index'], axis=1)

In [5]:
train_df

,toxic_comment,neutral_comment1
0,"и,чё,блядь где этот херой был до этого со свои...","Ну и где этот герой был,со своими доказательст..."
1,"О, а есть деанон этого петуха?","О, а есть деанон"
2,"херну всякую пишут,из-за этого лайка.долбоебизм.","Чушь всякую пишут, из- за этого лайка."
3,из за таких пидоров мы и страдаем,из за таких плохих людей мы и страдаем
4,гондон путинский он а не артист,"Человек Путина он, а не артист"
...,...,...
6943,"Блядь, пусть его уже закроют до конца его дней...","Пусть его уже закроют до конца его дней, он же..."
6944,твоя химия это тотальный пиздец(,твоя химия - это просто кошмар
6945,"меня изнасиловали, мудилка, а тебе пох(((","меня изнасиловали,а тебе всё равно ((("
6946,когда наплоюсь на пидораса и маньяка похож((((...,Когда напьюсь - на маньяка похож...


## My method of text processing

In [6]:
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50")

In [7]:
class DetoxDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __getitem__(self, idx):

        source = self.tokenizer(
            self.data.iloc[idx].toxic_comment,
            max_length=150,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer(
            self.data.iloc[idx].neutral_comment1,
            max_length=150,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        source["labels"] = target["input_ids"]

        return {k: v.squeeze(0) for k, v in source.items()}

    def __len__(self):
        return self.data.shape[0]

In [8]:
def paraphrase(
    text,
    model,
    tokenizer,
    n=None,
    max_length="auto",
    beams=5,
):
    texts = [text] if isinstance(text, str) else text
    inputs = tokenizer(texts, return_tensors="pt", padding=True)["input_ids"].to(
        model.device
    )

    if max_length == "auto":
        max_length = inputs.shape[1] + 10

    result = model.generate(
        inputs,
        num_return_sequences=n or 1,
        do_sample=False,
        temperature=1.,
        repetition_penalty=10.0,
        max_length=max_length,
        min_length=int(0.5 * max_length),
        num_beams=beams,
        # forced_bos_token_id=tokenizer.lang_code_to_id[tokenizer.tgt_lang],
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]

    if not n and isinstance(text, str):
        return texts[0]
    return texts[0]

In [9]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train_df, random_state=42, test_size=0.01)
trainset = DetoxDataset(train, tokenizer)
valset = DetoxDataset(val, tokenizer)

In [10]:
training_args = Seq2SeqTrainingArguments(
    output_dir="bart_detox",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=2,  # 8 is too much
    weight_decay=1e-5,
    num_train_epochs=3, # use 3 or 5 epochs here
    learning_rate=3e-5,
    evaluation_strategy="steps",
    save_strategy="no",
    save_total_limit=1,
    logging_steps=50,
    gradient_accumulation_steps=1,
)

In [11]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset=valset,
    tokenizer=tokenizer,
)

In [12]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a MBart50TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,8.295800,4.203672
100,1.799000,0.394062
150,1.516900,0.872687
200,0.324800,0.178024
250,0.173000,0.166070
300,0.156000,0.159486
350,1.119100,2.096885
400,1.253900,0.191931
450,0.155100,0.166987
500,0.157000,0.152549


TrainOutput(global_step=2580, training_loss=0.35964319983194043, metrics={'train_runtime': 4321.3227, 'train_samples_per_second': 4.775, 'train_steps_per_second': 0.597, 'total_flos': 6550276703846400.0, 'train_loss': 0.35964319983194043, 'epoch': 3.0})

Generate detoxifications and save them in a .txt file

In [14]:
test_df['toxic_comment'][2]

'актёр может и не плохой,но как человек - хуйло конченное.'

In [15]:
preds[2]

'актёр может и не плохой,но как человек - нет.'

In [16]:
preds = []
for i in tqdm(range(test_df.shape[0])):
    preds.append(paraphrase(test_df['toxic_comment'][i], model, tokenizer))

with open(f"predictions.txt", "w") as f:
    f.write("\n".join(preds))

100%|██████████| 875/875 [07:11<00:00,  2.03it/s]


In [19]:
!zip menace.zip predictions.txt

  adding: predictions.txt (deflated 68%)
